In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve,roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import Lasso,Ridge,RidgeCV,LassoCV

In [ ]:
data=pd.read_csv('https://raw.githubusercontent.com/dsrscientist/dataset3/main/weatherAUS.csv')
data.head()

In [ ]:
data=pd.read_csv('weatherAUS.csv')
data
#To be used in case raw file link of github isnt working

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.isna().sum()

Both imputation and encoding is needed

In [ ]:
data['MinTemp']=data['MinTemp'].fillna(data['MinTemp'].mean())
data['MaxTemp']=data['MaxTemp'].fillna(data['MaxTemp'].mean())
data['Rainfall']=data['Rainfall'].fillna(data['Rainfall'].mean())
data['Evaporation']=data['Evaporation'].fillna(data['Evaporation'].mean())
data['Sunshine']=data['Sunshine'].fillna(data['Sunshine'].mean())
data['WindGustSpeed']=data['WindGustSpeed'].fillna(data['WindGustSpeed'].mean())
data['WindSpeed9am']=data['WindSpeed9am'].fillna(data['WindSpeed9am'].mean())
data['WindSpeed3pm']=data['WindSpeed3pm'].fillna(data['WindSpeed3pm'].mean())
data['Humidity9am']=data['Humidity9am'].fillna(data['Humidity9am'].mean())
data['Humidity3pm']=data['Humidity3pm'].fillna(data['Humidity3pm'].mean())
data['Pressure9am']=data['Pressure9am'].fillna(data['Pressure9am'].mean())
data['Pressure3pm']=data['Pressure3pm'].fillna(data['Pressure3pm'].mean())
data['Cloud9am']=data['Cloud9am'].fillna(data['Cloud9am'].mean())
data['Cloud3pm']=data['Cloud3pm'].fillna(data['Cloud3pm'].mean())
data['Temp9am']=data['Temp9am'].fillna(data['Temp9am'].mean())
data['Temp3pm']=data['Temp3pm'].fillna(data['Temp3pm'].mean())

In [ ]:
data['WindGustDir']=data['WindGustDir'].fillna(data['WindGustDir'].mode()[0])
data['WindDir9am']=data['WindDir9am'].fillna(data['WindDir9am'].mode()[0])
data['WindDir3pm']=data['WindDir3pm'].fillna(data['WindDir3pm'].mode()[0])
data['RainToday']=data['RainToday'].fillna(data['RainToday'].mode()[0])
data['RainTomorrow']=data['RainTomorrow'].fillna(data['RainTomorrow'].mode()[0])

In [ ]:
data.isna().sum()

In [ ]:
data.info()

In [ ]:
lab_enc=LabelEncoder()

In [ ]:
print(data['Date'].value_counts().sort_values())
df=lab_enc.fit_transform(data['Date'])
pd.Series(df)
data['Date']=df

In [ ]:
print(data['Location'].value_counts().sort_values())
df=lab_enc.fit_transform(data['Location'])
pd.Series(df)
data['Location']=df

In [ ]:
print(data['WindGustDir'].value_counts().sort_values())
df=lab_enc.fit_transform(data['WindGustDir'])
pd.Series(df)
data['WindGustDir']=df

In [ ]:
print(data['WindDir9am'].value_counts().sort_values())
df=lab_enc.fit_transform(data['WindDir9am'])
pd.Series(df)
data['WindDir9am']=df

In [ ]:
print(data['WindDir3pm'].value_counts().sort_values())
df=lab_enc.fit_transform(data['WindDir3pm'])
pd.Series(df)
data['WindDir3pm']=df

In [ ]:
print(data['RainToday'].value_counts().sort_values())
df=lab_enc.fit_transform(data['RainToday'])
pd.Series(df)
data['RainToday']=df

In [ ]:
print(data['RainTomorrow'].value_counts().sort_values())
df=lab_enc.fit_transform(data['RainTomorrow'])
pd.Series(df)
data['RainTomorrow']=df

In [ ]:
plt.figure(figsize=(25,25),facecolor='white')
p=1
for column in data:
    if p <=24:
        ax=plt.subplot(5,5,p)
        sns.distplot(data[column])
        plt.xlabel(column,fontsize=20)
    p+=1
plt.tight_layout()

# 1 Whether it will rain tomorrow

In [ ]:
y=data['RainTomorrow']
x=data.drop(columns=['RainTomorrow'])

In [ ]:
plt.figure(figsize=(25,25),facecolor='white')
plotnumber=1
for column in x:
    if plotnumber<=21:
        ax=plt.subplot(7,3,plotnumber)
        plt.scatter(x[column],y)
        plt.xlabel(column,fontsize=20)
        plt.ylabel('RainTomorrow',fontsize=20)
    plotnumber+=1
plt.tight_layout()

In [ ]:
data_corr=data.corr().abs()
plt.figure(figsize=(20,20))
sns.heatmap(data_corr,annot=True,annot_kws={'size':10})
plt.show

In [ ]:
best_features=SelectKBest(score_func=f_classif,k='all')

fit=best_features.fit(x,y)

data_scores=pd.DataFrame(fit.scores_)

data_columns=pd.DataFrame(x.columns)

feature_scores=pd.concat([data_columns,data_scores],axis=1)

feature_scores.columns=['Feature_Name','Score']

print(feature_scores.nlargest(18,'Score'))

In [ ]:
for i, col in enumerate(data.columns):
    plt.figure(i)
    sns.lineplot(x='RainTomorrow',y=col, data=data)

Based on heatmap, lineplot and Selectk, we will move ahead with below features

In [ ]:
y=data['RainTomorrow']
x=data[['Cloud3pm','Sunshine','RainToday','Cloud9am','Humidity9am','Rainfall','Temp3pm','WindGustSpeed','Pressure9am','MaxTemp','Pressure3pm','MinTemp','WindSpeed9am','Evaporation','WindSpeed3pm','WindGustDir','WindDir9am']]

In [ ]:
plt.figure(figsize=(30,40),facecolor='white')
plotno=1
for column in x:
    if plotno<=20:
        ax=plt.subplot(5,4,plotno)
        sns.boxplot(data=x[column])
        plt.xlabel(column,fontsize=20)
    plotno+=1
plt.show()

We need to remove outliers from temp3pm, maxtemp, windspeed9am, windspeed3pm, humidity9am

In [ ]:
q1=data.quantile(0.25)
q3=data.quantile(0.75)
iqr=q3-q1

In [ ]:
MTh=(q3.MaxTemp + (1.5*iqr.MaxTemp))
index=np.where(data['MaxTemp']>MTh)
data=data.drop(data.index[index])
data.shape

In [ ]:
hml=(q3.Humidity9am - (1.5*iqr.Humidity9am))
index=np.where(data['Humidity9am']<ssl)
data=data.drop(data.index[index])
data.shape

In [ ]:
wsh=(q3.WindSpeed9am + (1.5*iqr.WindSpeed9am))
index=np.where(data['WindSpeed9am']>wsh)
data=data.drop(data.index[index])
data.shape

In [ ]:
wsh=(q3.WindSpeed3pm + (1.5*iqr.WindSpeed3pm))
index=np.where(data['WindSpeed3pm']>wsh)
data=data.drop(data.index[index])
data.shape

In [ ]:
y=data['RainTomorrow']
x=data[['Cloud3pm','Sunshine','RainToday','Cloud9am','Humidity9am','Rainfall','Temp3pm','WindGustSpeed','Pressure9am','MaxTemp','Pressure3pm','MinTemp','WindSpeed9am','Evaporation','WindSpeed3pm','WindGustDir','WindDir9am']]

In [ ]:
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x)
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.25,random_state=333)

In [ ]:
vif=pd.DataFrame()
vif['VIF']=[variance_inflation_factor(x_scaled,i) for i in range(x_scaled.shape[1])]
vif['Features']=x.columns
vif

In [ ]:
def metric_score(clf,x_train,x_test,y_train,y_test,train=True):
    if train:
        y_pred=clf.predict(x_train)
        print('\n----Train result----')
        print(f'Accuracy Score:{accuracy_score(y_train,y_pred)*100:.2f}%')
        
    elif train==False:
        pred=clf.predict(x_test)
        print('\n----Test result----')
        print(f'Accuracy Score:{accuracy_score(y_test,pred)*100:.2f}%')

        print('\n\n Test Classification report \n',classification_report(y_test,pred,digits=2))

In [ ]:
knn=KNeighborsClassifier()
knn.fit(x_train,y_train)

In [ ]:
metric_score(knn,x_train,x_test,y_train,y_test,train=True)
metric_score(knn,x_train,x_test,y_train,y_test,train=False)

In [ ]:
log_reg=LogisticRegression()
log_reg.fit(x_train,y_train)

In [ ]:
metric_score(log_reg,x_train,x_test,y_train,y_test,train=True)
metric_score(log_reg,x_train,x_test,y_train,y_test,train=False)

In [ ]:
rnd_clf=RandomForestClassifier()
rnd_clf.fit(x_train,y_train)

In [ ]:
metric_score(rnd_clf,x_train,x_test,y_train,y_test,train=True)
metric_score(rnd_clf,x_train,x_test,y_train,y_test,train=False)

In [ ]:
bag_knn=BaggingClassifier(KNeighborsClassifier(n_neighbors=9),
                          n_estimators=6,max_samples=0.75,
                          bootstrap=True,random_state=366,oob_score=True)

In [ ]:
bag_knn.fit(x_train,y_train)

In [ ]:
metric_score(bag_knn,x_train,x_test,y_train,y_test,train=True)
metric_score(bag_knn,x_train,x_test,y_train,y_test,train=False)

In [ ]:
svc=SVC()
svc.fit(x_train,y_train)

In [ ]:
metric_score(svc,x_train,x_test,y_train,y_test,train=True)
metric_score(svc,x_train,x_test,y_train,y_test,train=False)

In [ ]:
clf=DecisionTreeClassifier()
clf.fit(x_train,y_train)

In [ ]:
metric_score(clf,x_train,x_test,y_train,y_test,train=True)
metric_score(clf,x_train,x_test,y_train,y_test,train=False)

In [ ]:
grid_param={'criterion':['gini','entropy'],
           'max_depth':range(10,15),
           'min_samples_leaf':range(2,6),
           'min_samples_split':range(3,8),
           'max_leaf_nodes':range(5,10)}

In [ ]:
grid_search = GridSearchCV(estimator=clf,
                           param_grid=grid_param,
                           cv=5,
                           n_jobs=-1)

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
best_parameters=grid_search.best_params_
print(best_parameters)

In [ ]:
dclf=DecisionTreeClassifier(criterion='gini',max_depth= 10, max_leaf_nodes= 9, min_samples_leaf= 2, min_samples_split= 3)

In [ ]:
dclf.fit(x_train,y_train)

In [ ]:
metric_score(dclf,x_train,x_test,y_train,y_test,train=True)
metric_score(dclf,x_train,x_test,y_train,y_test,train=False)

Decision Tree amplified by Hyper parameter tuning is giving the best result

In [ ]:
from sklearn.model_selection import KFold,cross_val_score
cross_val_score(dclf,x_scaled,y,cv=10)

In [ ]:
y_pred=log_reg.predict(x_test)
confusion_mat = confusion_matrix(y_test,y_pred)
print(confusion_mat)

In [ ]:
pickle.dump(dclf,open('Rain prediction','wb'))

# 2 Amount of Rainfall

In [ ]:
x=data.drop(columns=['Rainfall'])
y=data['Rainfall']

In [ ]:
plt.figure(figsize=(15,15),facecolor='white')
plotnumber=1
for column in x:
    if plotnumber<=25:
        ax=plt.subplot(6,4,plotnumber)
        plt.scatter(x[column],y)
        plt.xlabel(column,fontsize=10)
        plt.ylabel('Rainfall',fontsize=10)
    plotnumber+=1
plt.tight_layout()

In [ ]:
for i, col in enumerate(data.columns):
    plt.figure(i)
    sns.lineplot(y='Rainfall',x=col, data=data)

In [ ]:
best_features=SelectKBest(score_func=f_classif,k='all')

fit=best_features.fit(x,y)

data_scores=pd.DataFrame(fit.scores_)

data_columns=pd.DataFrame(x.columns)

feature_scores=pd.concat([data_columns,data_scores],axis=1)

feature_scores.columns=['Feature_Name','Score']

print(feature_scores.nlargest(19,'Score'))

In [ ]:
y=data['Rainfall']
x=data[['RainToday','Evaporation','Humidity9am','Humidity3pm','RainTomorrow','Cloud9am','Sunshine','Cloud3pm','Temp3pm','MaxTemp','Pressure9am','WindGustSpeed','WindDir9am','WindSpeed3pm','WindGustDir','Pressure3pm','WindSpeed9am','Temp9am']]

In [ ]:
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x)

In [ ]:
vif=pd.DataFrame()
vif['vif']=[variance_inflation_factor(x_scaled,i) for i in range(x_scaled.shape[1])]
vif['Features']=x.columns
vif

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.22,random_state=44)

In [ ]:
reg=LinearRegression()
reg.fit(x_train,y_train)

In [ ]:
reg.score(x_train,y_train)

In [ ]:
reg.score(x_test,y_test)

In [ ]:
lasscv=LassoCV(alphas=None,max_iter=300,normalize=True)
lasscv.fit(x_train,y_train)

In [ ]:
alpha=lasscv.alpha_
alpha

In [ ]:
lasso_reg=Lasso(alpha)
lasso_reg.fit(x_train,y_train)

In [ ]:
lasso_reg.score(x_train,y_train)

In [ ]:
lasso_reg.score(x_test,y_test)

In [ ]:
ridgecv=RidgeCV(alphas=np.arange(0.002,0.1,0.02),normalize=True)
ridgecv.fit(x_train,y_train)

In [ ]:
ridgecv.alpha_

In [ ]:
ridge_model=Ridge(alpha=ridgecv.alpha_)
ridge_model.fit(x_train,y_train)

In [ ]:
ridge_model.score(x_test,y_test)

In [ ]:
y_pred=reg.predict(x_test)

In [ ]:
plt.scatter(y_test,y_pred)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Model Predicted')
plt.show()

In [ ]:
y_pred=reg.predict(x_test)

In [ ]:
mean_absolute_error(y_test,y_pred)

In [ ]:
mean_squared_error(y_test,y_pred)

In [ ]:
np.sqrt(mean_squared_error(y_test,y_pred))

In [ ]:
pickle.dump(lasso_reg,open('Rainfall amount','wb'))